In [30]:
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv("data/OMNI_dataset_20252026.csv")

print(f"Initial dataset shape: {df.shape}")
print(f"Initial columns: {df.columns.tolist()}")

# Create Datetime column
df['Datetime'] = pd.to_datetime(df['Year'].astype(str) + df['DOY'].astype(str), format='%Y%j') \
                 + pd.to_timedelta(df['Hour'], unit='h')

# Set Datetime as index (helpful for time-series)
df.set_index('Datetime', inplace=True)

Initial dataset shape: (9000, 10)
Initial columns: ['Year', 'DOY', 'Hour', 'Bz_GSM_nT', 'Proton_Temp_K', 'Proton_Density_cm3', 'Solar_Wind_Speed_kms', 'Flow_Pressure_nPa', 'Kp_x10', 'Kp']


In [31]:
# ========== HANDLE MISSING VALUES ==========
print("\n" + "="*50)
print("Missing values before cleaning:")
print(df.isnull().sum())

df.interpolate(method='linear', limit=3, inplace=True)
df.fillna(method='ffill', limit=2, inplace=True)
df.fillna(method='bfill', limit=2, inplace=True)

print("\nMissing values after cleaning:")
print(df.isnull().sum())


Missing values before cleaning:
Year                    0
DOY                     0
Hour                    0
Bz_GSM_nT               0
Proton_Temp_K           0
Proton_Density_cm3      0
Solar_Wind_Speed_kms    0
Flow_Pressure_nPa       0
Kp_x10                  0
Kp                      0
dtype: int64

Missing values after cleaning:
Year                    0
DOY                     0
Hour                    0
Bz_GSM_nT               0
Proton_Temp_K           0
Proton_Density_cm3      0
Solar_Wind_Speed_kms    0
Flow_Pressure_nPa       0
Kp_x10                  0
Kp                      0
dtype: int64


/var/folders/sz/6znz8j351872v17bd_n9xhg80000gn/T/ipykernel_319/3422320694.py:7: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', limit=2, inplace=True)
/var/folders/sz/6znz8j351872v17bd_n9xhg80000gn/T/ipykernel_319/3422320694.py:8: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', limit=2, inplace=True)


In [32]:
# ========== LAG FEATURES ==========
lags = [1, 2, 3, 6]

for lag in lags:
    df[f'Bz_lag_{lag}'] = df['Bz_GSM_nT'].shift(lag)
    df[f'V_lag_{lag}'] = df['Solar_Wind_Speed_kms'].shift(lag)
    df[f'D_lag_{lag}'] = df['Proton_Density_cm3'].shift(lag)
    df[f'Kp_lag_{lag}'] = df['Kp'].shift(lag)

print(f"\nAfter lag features: {df.shape}")


After lag features: (9000, 26)


In [33]:
# ========== ROLLING STATISTICS ==========
# FIX: Remove window=1 for std (needs at least 2 values)
# Use windows [3, 6] for std, keep [1, 3, 6] for mean/min/max

windows_all = [1, 3, 6]      # For mean, min, max
windows_std = [3, 6]          # For std (skip window=1)

for w in windows_all:
    # Mean
    df[f'Bz_roll_mean_{w}'] = df['Bz_GSM_nT'].rolling(w).mean()
    df[f'V_roll_mean_{w}'] = df['Solar_Wind_Speed_kms'].rolling(w).mean()
    df[f'D_roll_mean_{w}'] = df['Proton_Density_cm3'].rolling(w).mean()
    
    # Min/Max
    df[f'Bz_roll_min_{w}'] = df['Bz_GSM_nT'].rolling(w).min()
    df[f'V_roll_max_{w}'] = df['Solar_Wind_Speed_kms'].rolling(w).max()
    df[f'D_roll_max_{w}'] = df['Proton_Density_cm3'].rolling(w).max()

# Std (only for windows >= 2)
for w in windows_std:
    df[f'Bz_roll_std_{w}'] = df['Bz_GSM_nT'].rolling(w).std()
    df[f'V_roll_std_{w}'] = df['Solar_Wind_Speed_kms'].rolling(w).std()
    df[f'D_roll_std_{w}'] = df['Proton_Density_cm3'].rolling(w).std()


In [34]:
# ========== RATE OF CHANGE ==========
df['dBz_dt'] = df['Bz_GSM_nT'].diff()
df['dV_dt'] = df['Solar_Wind_Speed_kms'].diff()
df['dD_dt'] = df['Proton_Density_cm3'].diff()

print(f"After rate of change: {df.shape}")

# ========== PHYSICS-BASED FEATURES ==========
df['Bz_times_V'] = df['Bz_GSM_nT'] * df['Solar_Wind_Speed_kms']
df['Bz_times_D'] = df['Bz_GSM_nT'] * df['Proton_Density_cm3']
df['V2_times_D'] = (df['Solar_Wind_Speed_kms']**2) * df['Proton_Density_cm3']

# Electric field proxy
df['Electric_Field_Proxy'] = df['Solar_Wind_Speed_kms'] * abs(df['Bz_GSM_nT'])
df['Electric_Field_South'] = df.apply(
    lambda row: row['Solar_Wind_Speed_kms'] * abs(row['Bz_GSM_nT']) 
    if row['Bz_GSM_nT'] < 0 else 0, axis=1
)

# Alfvén Mach number proxy
df['Alfven_Mach_Proxy'] = df['Solar_Wind_Speed_kms'] / (abs(df['Bz_GSM_nT']) + 1)

print(f"After physics features: {df.shape}")

# ========== SOUTHWARD Bz FEATURES ==========
df['Bz_negative'] = (df['Bz_GSM_nT'] < 0).astype(int)
df['Bz_neg_duration'] = df['Bz_negative'].groupby(
    (df['Bz_negative'] != df['Bz_negative'].shift()).cumsum()
).cumsum()

df['Bz_southward_strength'] = df['Bz_GSM_nT'].apply(lambda x: abs(x) if x < 0 else 0)

print(f"After southward Bz features: {df.shape}")

# ========== TIME-BASED FEATURES ==========
df.reset_index(inplace=True)
df['Hour_of_Day'] = df['Datetime'].dt.hour
df['Day_of_Year'] = df['Datetime'].dt.dayofyear
df['Month'] = df['Datetime'].dt.month

print(f"After time features: {df.shape}")

After rate of change: (9000, 53)
After physics features: (9000, 59)
After southward Bz features: (9000, 62)
After time features: (9000, 66)


In [39]:
# ========== FINAL CLEANUP (WITH DIAGNOSTICS) ==========
print("\n" + "="*50)
print("CLEANUP DIAGNOSTICS:")
print(f"Rows before cleanup: {len(df)}")

# Check how many NaNs exist
print(f"\nTotal NaN values: {df.isnull().sum().sum()}")
print("\nColumns with NaN values:")
print(df.isnull().sum()[df.isnull().sum() > 0])

# Check the first cleanup step
print(f"\nRows that would be dropped by dropna(): {df.isnull().any(axis=1).sum()}")

# MODIFIED CLEANUP - Drop only the first few rows affected by lag/rolling
# Instead of dropping ALL rows with ANY NaN, drop only first 6 rows (max lag)
df_clean = df.iloc[6:].copy()  # Skip first 6 rows (max lag used)

print(f"Rows after dropping first 6: {len(df_clean)}")

# Check for invalid solar wind speed
invalid_speed = (df_clean['Solar_Wind_Speed_kms'] <= 0).sum()
print(f"Rows with invalid solar wind speed (≤0): {invalid_speed}")

df_clean = df_clean[df_clean['Solar_Wind_Speed_kms'] >= 0]
print(f"Rows after removing invalid speed: {len(df_clean)}")

# Check for invalid density
invalid_density = (df_clean['Proton_Density_cm3'] <= 0).sum()
print(f"Rows with invalid density (≤0): {invalid_density}")

df_clean = df_clean[df_clean['Proton_Density_cm3'] >= 0]
print(f"Rows after removing invalid density: {len(df_clean)}")

# Final NaN check
print(f"\nFinal NaN check: {df_clean.isnull().sum().sum()} NaNs remaining")

print(f"\nFINAL DATASET SHAPE: {df_clean.shape}")
print(f"Total features: {len(df_clean.columns)}")


CLEANUP DIAGNOSTICS:
Rows before cleanup: 9000

Total NaN values: 853

Columns with NaN values:
Solar_Wind_Speed_kms     61
Bz_lag_1                  1
V_lag_1                  62
D_lag_1                   1
Kp_lag_1                  1
Bz_lag_2                  2
V_lag_2                  63
D_lag_2                   2
Kp_lag_2                  2
Bz_lag_3                  3
V_lag_3                  64
D_lag_3                   3
Kp_lag_3                  3
Bz_lag_6                  6
V_lag_6                  67
D_lag_6                   6
Kp_lag_6                  6
V_roll_mean_1            61
V_roll_max_1             61
Bz_roll_mean_3            2
V_roll_mean_3            30
D_roll_mean_3             2
Bz_roll_min_3             2
V_roll_max_3            108
D_roll_max_3              2
Bz_roll_mean_6            5
V_roll_mean_6            15
D_roll_mean_6             5
Bz_roll_min_6             5
V_roll_max_6            171
D_roll_max_6              5
Bz_roll_std_3             2
V_roll_

In [40]:
df.replace([9999, 99999, 999999, 1e5], np.nan, inplace=True)

In [41]:
# ========== SAVE ==========
if len(df_clean) > 0:
    df_clean.to_csv("data/OMNI_dataset_feature_engineered_20252026.csv", index=False)
    print("\n✅ Feature engineered dataset saved successfully!")
    print(f"Saved {len(df_clean)} rows with {len(df_clean.columns)} columns")
else:
    print("\n❌ ERROR: No data to save! All rows were filtered out.")
    print("\nPlease check your original dataset for issues.")


✅ Feature engineered dataset saved successfully!
Saved 8933 rows with 66 columns
